## Capstone Project

In [30]:
# install packages
!pip install pycaret
!pip install unidecode

  Using cached pycaret-2.2.2-py3-none-any.whl (249 kB)
  Using cached lightgbm-3.1.1-py2.py3-none-win_amd64.whl (754 kB)
  Using cached umap-learn-0.4.6.tar.gz (69 kB)
  Using cached xgboost-1.3.0.post0-py3-none-win_amd64.whl (95.2 MB)
  Using cached pandas_profiling-2.9.0-py2.py3-none-any.whl (258 kB)
  Using cached kmodes-0.10.2-py2.py3-none-any.whl (18 kB)
  Using cached spacy-2.3.5-cp38-cp38-win_amd64.whl (9.7 MB)
  Using cached preshed-3.0.5-cp38-cp38-win_amd64.whl (112 kB)
  Using cached plac-1.1.3-py2.py3-none-any.whl (20 kB)
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97717 sha256=c868f1893a3e2f9dd33e13a8ba58d961a28ea3f8c430aec938f0f49e95f7df7e
  Stored in directory: c:\users\prade\appdata\local\pip\cache\wheels\31\8c\a0\24a443892f2134e691d59c8c6c35e19821e02f85e49871f8fd
  Created wheel for cufflinks: filename=cufflinks-0.17.3-py3-none-any.whl size=68727 sha256=a190b89e8f757ef15cd2c4fc57c95c63aa3bd877618d9db2ec0a70493dcc0d70
  Stored in direc

ERROR: Could not install packages due to an EnvironmentError: [WinError 32] The process cannot access the file because it is being used by another process: 'e:\\Users\\prade\\anaconda3\\Lib\\site-packages\\catboost\\_catboost.pyd'
Consider using the `--user` option or check the permissions.



  Created wheel for suod: filename=suod-0.0.4-py3-none-any.whl size=2167162 sha256=ae11feafe94c6085800de9f439fef5bee097885bce58c7b9cf34ba06c10af404
  Stored in directory: c:\users\prade\appdata\local\pip\cache\wheels\ad\1a\f5\973bd291c9772840f1ae9a1e40e4f2522b786db241ad570e91
  Created wheel for smart-open: filename=smart_open-4.0.1-py3-none-any.whl size=108251 sha256=0fcda1532d858d2d8d0b2f3bd65630de76f8d1b491359bf7a88e9c7a1a36437d
  Stored in directory: c:\users\prade\appdata\local\pip\cache\wheels\8c\f9\f4\4ddd9ddee3488f48be20e9bf3108961f03ae23da29b7ed26d1
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27091 sha256=64ab16213e7014561de3f65bbb3a37312faecb31f54a633a780ccc6589491a70
  Stored in directory: c:\users\prade\appdata\local\pip\cache\wheels\23\14\6e\4be5bfeeb027f4939a01764b48edd5996acf574b0913fe5243
Successfully built pyLDAvis cufflinks umap-learn pyod prometheus-flask-exporter databricks-cli alembic combo suod smart-open htmlmin
  Attempting uninsta

  Using cached pyLDAvis-2.1.2.tar.gz (1.6 MB)
  Using cached scikit_plot-0.3.7-py3-none-any.whl (33 kB)
  Using cached cufflinks-0.17.3.tar.gz (81 kB)
  Using cached textblob-0.15.3-py2.py3-none-any.whl (636 kB)
  Using cached imbalanced_learn-0.7.0-py3-none-any.whl (167 kB)
  Using cached wordcloud-1.8.1-cp38-cp38-win_amd64.whl (155 kB)
  Using cached yellowbrick-1.2-py3-none-any.whl (269 kB)
  Using cached pyod-0.8.4.tar.gz (98 kB)
  Using cached funcy-1.15-py2.py3-none-any.whl (32 kB)
  Using cached GitPython-3.1.11-py3-none-any.whl (159 kB)
  Using cached prometheus_flask_exporter-0.18.1.tar.gz (21 kB)
  Using cached protobuf-3.14.0-py2.py3-none-any.whl (173 kB)
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
  Using cached waitress-1.4.4-py2.py3-none-any.whl (58 kB)
  Using cached azure_storage_blob-12.6.0-py2.py3-none-any.whl (328 kB)
  Using cached sqlparse-0.4.1-py3-none-any.whl (42 kB)
  Using cached databricks-cli-0.14.1.tar.gz (54 kB)
  Using cached doc

In [31]:
# download corpus of stop words
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prade\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
# import modules
import numpy as np
import pandas as pd
import plotly.express as px
import scipy.stats as ss
import unidecode
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [33]:
# load the data
data = pd.read_csv("pet food data.csv")

There are 684 entries that do not have price data, 548 entries that do not have an ingredients list, 755 entries that do not have a total package size, and 747 entries that do not have a unit package size.

In [34]:
# get a description of the unprocessed data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8754 entries, 0 to 8753
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  8754 non-null   object 
 1   category              8754 non-null   object 
 2   sub_category          8754 non-null   object 
 3   product               8754 non-null   object 
 4   variant               8754 non-null   object 
 5   description           8754 non-null   object 
 6   ingredients_stdlist   8206 non-null   object 
 7   price_usd             8070 non-null   float64
 8   company_parent        8754 non-null   object 
 9   company               8754 non-null   object 
 10  brand                 8754 non-null   object 
 11  total_pack_size_ml_g  7999 non-null   float64
 12  unit_pack_size_ml_g   8007 non-null   float64
dtypes: float64(3), object(10)
memory usage: 889.2+ KB


In [35]:
# determine if there are any duplicate rows
np.sum(data.duplicated())

0

### Initial Cleaning for Visualization

In [36]:
# drop the 900 entries that do not have price data or an ingredients list
data.dropna(inplace=True, subset=["price_usd", "ingredients_stdlist"])

# remove 1 outlier price with invalid pack size
data = data[data.price_usd < 50]

# drop 32 therapeutic supplements 
data = data.loc[~data.ingredients_stdlist.str.startswith("active"), :]

# reset the index
data.reset_index(drop=True, inplace=True)

In [37]:
# transform the string dates into datetime objects and create a year column
data.loc[:, "date"] = pd.to_datetime(data.date.str[:10])
data["year"] = data.date.apply(lambda x: x.year)

In [38]:
# drop the "category" column since it contains one value "Pet Food" and
# the "variant" column since it is equivalent to the "product" column
data.drop(columns=["category", "product"], inplace=True)

In [39]:
# transform the "sub_category" column into separate columns and drop "sub_category"
data["pet_type"] = data.sub_category.apply(lambda x: "cat" if ("Cat" in x) else "dog")
data["meal_type"] = data.sub_category.apply(lambda x: "primary" if ("Food" in x) else "treats")
data["food_type"] = data.sub_category.apply(lambda x: "dry" if ("Dry" in x) else "wet")
data.drop(columns="sub_category", inplace=True)

In [40]:
# create a function to replace dicritic characters with ASCII characters
def replace_dicritic(string):
    return unidecode.unidecode(string)

In [41]:
# replace the dicritic characters in "company_parent", "company", "brand",
# "description", and "ingredients_stdlist"
data.loc[:, "company_parent"] = data.company_parent.apply(replace_dicritic)
data.loc[:, "company"] = data.company.apply(replace_dicritic)
data.loc[:, "brand"] = data.brand.apply(replace_dicritic)
data.loc[:, "description"] = data.description.apply(replace_dicritic)
data.loc[:, "ingredients_stdlist"] = data.ingredients_stdlist.apply(replace_dicritic)

In [42]:
# get a description of the cleaned data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7822 entries, 0 to 7821
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  7822 non-null   datetime64[ns]
 1   variant               7822 non-null   object        
 2   description           7822 non-null   object        
 3   ingredients_stdlist   7822 non-null   object        
 4   price_usd             7822 non-null   float64       
 5   company_parent        7822 non-null   object        
 6   company               7822 non-null   object        
 7   brand                 7822 non-null   object        
 8   total_pack_size_ml_g  7526 non-null   float64       
 9   unit_pack_size_ml_g   7534 non-null   float64       
 10  year                  7822 non-null   int64         
 11  pet_type              7822 non-null   object        
 12  meal_type             7822 non-null   object        
 13  food_type         

### Visualizations

In [43]:
# get the observations for Colgate only
colgate_only = data[data.company_parent.apply(lambda x: "colgate" in x.lower())]

In [44]:
# perform a Box-Cox transformation on price_usd
box_cox = ss.boxcox(data.price_usd)

Boxplots

In [45]:
# boxplot for cat vs dog
fig1 = px.box(data, x="pet_type", y="price_usd")
fig2 = px.box(colgate_only, x="pet_type", y="price_usd", category_orders={"pet_type": ["cat", "dog"]})

# boxplot for primary food vs treats
fig3 = px.box(data, x="meal_type", y="price_usd")
fig4 = px.box(colgate_only, x="meal_type", y="price_usd")

# boxplot for dry vs wet food
fig5 = px.box(data, x="food_type", y="price_usd")
fig6 = px.box(colgate_only, x="food_type", y="price_usd", category_orders={"food_type": ["dry", "wet"]})

# boxplot for year
fig7 = px.box(data, x="year", y="price_usd")

Histograms

In [46]:
# histogram of price
fig8 = px.histogram(data.price_usd)

# histogram of price with Box-Cox transformation
fig9 = px.histogram(box_cox[0], nbins=50)

Scatter Plots

In [47]:
# scatter plot of price vs unit pack size
fig10 = px.scatter(data.loc[~data.unit_pack_size_ml_g.isna(), :], x="unit_pack_size_ml_g", y="price_usd", trendline="ols", trendline_color_override="orange", range_y=[-2, 37])

### Text Transformations

In [48]:
# get the first word in every ingredient list
first_ingredient = data.ingredients_stdlist.str.split(",").apply(lambda x: x[0])
first_ingredient = first_ingredient.str.split(" ").apply(lambda x: x[0])
first_ingredient = first_ingredient.str.split(":").apply(lambda x: x[0].lower())

In [49]:
# look at the top 10 words by frequency count
first_ingredient.value_counts()[:10]

chicken    1890
wheat       692
beef        611
corn        420
water       351
turkey      291
salmon      249
lamb        228
rice        210
cowhide     159
Name: ingredients_stdlist, dtype: int64

In [50]:
# create a function that filters the first_ingredient Series to
# 5 possible words
def first_word(string):
    if string == "chicken" or string == "wheat" or string == "beef" or string == "corn":
        return string
    else:
        return "other"

In [51]:
# add a first_ingredient column to the dataset
data["first_ingredient"] =  first_ingredient.apply(first_word)

TFIDF

During testing, we discovered that using a TFIDF matrix with more than ~400 features in our regression models caused instability, resulting in errors that were $10^6$ or greater. To improve performance, we set max_features=400 in our TfidfVectorizer().

In [52]:
# instantiate the TFIDF vectorizer
vectorizer = TfidfVectorizer(stop_words=stopwords.words("english"), max_features=400)

# transform the product descriptions in a TFIDF matrix
vectors = vectorizer.fit_transform(data.description)

# convert the TFIDF matrix into a dataframe
tfidf = pd.DataFrame(vectors.todense().tolist(), columns=vectorizer.get_feature_names())

Convert categorical variables to one-hot vectors

In [53]:
# get all of the categorical variables in one dataframe
cat_vars = data[["pet_type", "meal_type", "food_type", "first_ingredient"]]

# transform the categorical variables into one-hot vectors
one_hot = pd.get_dummies(cat_vars, prefix="category")

### Combine all of the engineered features and create training and testing sets

In [54]:
# combine the 1hot dataframe with the TFIDF dataframe
X = pd.concat([one_hot, tfidf], axis=1)

# create a target variable for the Box-Cox transformed prices
y = box_cox[0]

# create stratified training and testing sets
# prices box-cox
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, stratify=data.pet_type, random_state=38)

# original prices
X_train_org, X_test_org, y_train_org, y_test_org = train_test_split(X, data.price_usd, train_size=0.8, stratify=data.pet_type, random_state=38)

In [55]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6257, 411), (1565, 411), (6257,), (1565,))

In [56]:
X_train_org.shape, X_test_org.shape, y_train_org.shape, y_test_org.shape

((6257, 411), (1565, 411), (6257,), (1565,))

### pycaret test

In [57]:
from pycaret.regression import *

In [58]:
data2 = X.copy()

In [59]:
# get the transformed price data using box cox 
data2["price"] = box_cox[0]

In [60]:
exp_reg101 = setup(data = data2, target="price", data_split_stratify=["category_dog"], use_gpu=True)

,Description,Value
0,session_id,2725
1,Target,price
2,Original Data,"(7822, 412)"
3,Missing Values,False
4,Numeric Features,411
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(5475, 408)"


In [61]:
# get the best models with 5 fold cross validation sorted by root mean squared error
best = compare_models(fold=5, sort="RMSE")

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.5569,0.5911,0.7681,0.6760,0.2809,1.2906,2.4860
rf,Random Forest Regressor,0.5915,0.6281,0.7921,0.6558,0.2925,1.3732,2.3440
lightgbm,Light Gradient Boosting Machine,0.6018,0.6295,0.7928,0.6550,0.2920,1.4126,0.7840
catboost,CatBoost Regressor,0.6121,0.6420,0.8008,0.6482,0.2959,1.4983,19.1700
xgboost,Extreme Gradient Boosting,0.6240,0.6783,0.8232,0.6282,0.3029,1.4619,1.6080
gbr,Gradient Boosting Regressor,0.6583,0.7156,0.8457,0.6079,0.3112,1.6674,4.7300
br,Bayesian Ridge,0.6727,0.7662,0.8748,0.5802,0.3197,1.6896,0.1640
ridge,Ridge Regression,0.6741,0.7709,0.8775,0.5776,0.3203,1.6938,0.0420
lr,Linear Regression,0.6829,0.7923,0.8896,0.5659,0.3237,1.7292,0.0720
huber,Huber Regressor,0.6730,0.8022,0.8951,0.5605,0.3255,1.6030,2.2840
